https://colab.research.google.com/github/bentrevett/pytorch-pos-tagging/blob/master/1%20-%20BiLSTM%20for%20PoS%20Tagging.ipynb#scrollTo=6QHsOPobm93C


https://github.com/bentrevett/pytorch-pos-tagging

https://wikidocs.net/66747

이 시리즈에서는 PyTorch 및 TorchText를 사용하여 입력 시퀀스의 모든 요소에 대한 출력을 생성하는 모델을 구축 하여 입력 텍스트의 각 토큰에 대한 품사(Pos)를 출력한다. 이를 통해 객체명 인식 (NER)에 사용될 수 있다. 객체명 인식을 포털 사이트에 적용을 하게 되면 품사(Pos)가 무엇인지에 따라 더 정확한 결과를 도출해낼 수 있다.https://blog.est.ai/2020/10/ner/

이 노트북에서는 UDPOS (Universal Dependencies English Web Treebank) 데이터 세트를 사용하여 품사(Pos) 태그를 예측하기 위해 다 계층 양방향 LSTM (BiLSTM)을 구현할 것이다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import spacy
import numpy as np

import time
import random

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministric = True

이 데이터 세트에는 실제로 UD (Universal Dependency)(보편적 의존성) 태그와 PTB (Penn Treebank)(펜트리뱅크https://codedragon.tistory.com/9553) 태그의 두 가지 다른 태그 세트가 있습니다. 우리는 UD 태그에서만 모델을 훈련시킬 것이지만 대신에 어떻게 사용될 수 있는지 보여주기 위해 PTB 태그를 로드 할 것입니다.

UD_TAGS는 UD 태그 처리 방법을 처리한다. 나중에 구축 할 TEXT 어휘에는 알 수 없는 토큰, 즉 우리 어휘 내에없는 토큰이 포함된다. 그러나 가능한 태그의 유한 세트를 다루기 때문에 알 수 없는 태그는 없다.

PTB_TAGS도 UD_TAGS와 비슷하지만 PTB 태그를 처리한다. 

In [3]:
TEXT = data.Field(lower=True)
UD_TAGS = data.Field(unk_token = None)
PTB_TAGS = data.Field(unk_token = None)

In [4]:
fields = (('text', TEXT), ('udtags', UD_TAGS), ('ptbtags', PTB_TAGS))

In [5]:
train_data, valid_data, test_data = datasets.UDPOS.splits(fields)

In [6]:
print(f'Number of training examples : {len(train_data)}')
print(f'Number of validation examples : {len(valid_data)}')
print(f'Number of testing examples : {len(test_data)}')

Number of training examples : 12543
Number of validation examples : 2002
Number of testing examples : 2077


In [7]:
print(vars(train_data.examples[0]))

{'text': ['al', '-', 'zaman', ':', 'american', 'forces', 'killed', 'shaikh', 'abdullah', 'al', '-', 'ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'qaim', ',', 'near', 'the', 'syrian', 'border', '.'], 'udtags': ['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT'], 'ptbtags': ['NNP', 'HYPH', 'NNP', ':', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'NNP', 'HYPH', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', '.']}


In [8]:
print(vars(train_data.examples[0])['text'])

['al', '-', 'zaman', ':', 'american', 'forces', 'killed', 'shaikh', 'abdullah', 'al', '-', 'ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'qaim', ',', 'near', 'the', 'syrian', 'border', '.']


In [9]:
print(vars(train_data.examples[0])['udtags'])

['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']


In [10]:
print(vars(train_data.examples[0])['ptbtags'])

['NNP', 'HYPH', 'NNP', ':', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'NNP', 'HYPH', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', '.']


In [11]:
MIN_FREQ = 2
TEXT.build_vocab(train_data, min_freq = MIN_FREQ, vectors = 'glove.6B.100d', unk_init=torch.Tensor.normal_)

In [12]:
UD_TAGS.build_vocab(train_data)
PTB_TAGS.build_vocab(train_data)

In [13]:
print(f'Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}')
print(f'Unique tokens in UD_TAG vocabulary: {len(UD_TAGS.vocab)}')
print(f'Unique tokens in PTB_TAG vocabulary: {len(PTB_TAGS.vocab)}')

Unique tokens in TEXT vocabulary: 8866
Unique tokens in UD_TAG vocabulary: 18
Unique tokens in PTB_TAG vocabulary: 51


In [14]:
print(TEXT.vocab.freqs.most_common(20)) # 가장 많이 쓰이는 text 상위 20개

[('the', 9076), ('.', 8640), (',', 7021), ('to', 5137), ('and', 5002), ('a', 3782), ('of', 3622), ('i', 3379), ('in', 3112), ('is', 2239), ('you', 2156), ('that', 2036), ('it', 1850), ('for', 1842), ('-', 1426), ('have', 1359), ('"', 1296), ('on', 1273), ('was', 1244), ('with', 1216)]


In [15]:
print(UD_TAGS.vocab.itos) # itos : 숫자 -> text

['<pad>', 'NOUN', 'PUNCT', 'VERB', 'PRON', 'ADP', 'DET', 'PROPN', 'ADJ', 'AUX', 'ADV', 'CCONJ', 'PART', 'NUM', 'SCONJ', 'X', 'INTJ', 'SYM']


In [16]:
print(PTB_TAGS.vocab.itos)

['<pad>', 'NN', 'IN', 'DT', 'NNP', 'PRP', 'JJ', 'RB', '.', 'VB', 'NNS', ',', 'CC', 'VBD', 'VBP', 'VBZ', 'CD', 'VBN', 'VBG', 'MD', 'TO', 'PRP$', '-RRB-', '-LRB-', 'WDT', 'WRB', ':', '``', "''", 'WP', 'RP', 'UH', 'POS', 'HYPH', 'JJR', 'NNPS', 'JJS', 'EX', 'NFP', 'GW', 'ADD', 'RBR', '$', 'PDT', 'RBS', 'SYM', 'LS', 'FW', 'AFX', 'WP$', 'XX']


In [17]:
print(UD_TAGS.vocab.freqs.most_common())

[('NOUN', 34781), ('PUNCT', 23679), ('VERB', 23081), ('PRON', 18577), ('ADP', 17638), ('DET', 16285), ('PROPN', 12946), ('ADJ', 12477), ('AUX', 12343), ('ADV', 10548), ('CCONJ', 6707), ('PART', 5567), ('NUM', 3999), ('SCONJ', 3843), ('X', 847), ('INTJ', 688), ('SYM', 599)]


In [18]:
print(PTB_TAGS.vocab.freqs.most_common())

[('NN', 26915), ('IN', 20724), ('DT', 16817), ('NNP', 12449), ('PRP', 12193), ('JJ', 11591), ('RB', 10831), ('.', 10317), ('VB', 9476), ('NNS', 8438), (',', 8062), ('CC', 6706), ('VBD', 5402), ('VBP', 5374), ('VBZ', 4578), ('CD', 3998), ('VBN', 3967), ('VBG', 3330), ('MD', 3294), ('TO', 3286), ('PRP$', 3068), ('-RRB-', 1008), ('-LRB-', 973), ('WDT', 948), ('WRB', 869), (':', 866), ('``', 813), ("''", 785), ('WP', 760), ('RP', 755), ('UH', 689), ('POS', 684), ('HYPH', 664), ('JJR', 503), ('NNPS', 498), ('JJS', 383), ('EX', 359), ('NFP', 338), ('GW', 294), ('ADD', 292), ('RBR', 276), ('$', 258), ('PDT', 175), ('RBS', 169), ('SYM', 156), ('LS', 117), ('FW', 93), ('AFX', 48), ('WP$', 15), ('XX', 1)]


In [19]:
# 품사(Pos)들이 얼마나 쓰였는지 %로 확인 하기 위한 함수
def tag_percentage(tag_counts):
    total_count = sum([count for tag, count in tag_counts])
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]
    return tag_counts_percentages

In [20]:
print('Tag\t\tCount\t\tPercentage\n')

for tag, count, percent in tag_percentage(UD_TAGS.vocab.freqs.most_common()):
    print(f'{tag}\t\t{count}\t\t{percent*100:4.1f}%')

Tag		Count		Percentage

NOUN		34781		17.0%
PUNCT		23679		11.6%
VERB		23081		11.3%
PRON		18577		 9.1%
ADP		17638		 8.6%
DET		16285		 8.0%
PROPN		12946		 6.3%
ADJ		12477		 6.1%
AUX		12343		 6.0%
ADV		10548		 5.2%
CCONJ		6707		 3.3%
PART		5567		 2.7%
NUM		3999		 2.0%
SCONJ		3843		 1.9%
X		847		 0.4%
INTJ		688		 0.3%
SYM		599		 0.3%


In [21]:
print("Tag\t\tCount\t\tPercentage\n")

for tag, count, percent in tag_percentage(PTB_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t\t{count}\t\t{percent*100:4.1f}%")

Tag		Count		Percentage

NN		26915		13.2%
IN		20724		10.1%
DT		16817		 8.2%
NNP		12449		 6.1%
PRP		12193		 6.0%
JJ		11591		 5.7%
RB		10831		 5.3%
.		10317		 5.0%
VB		9476		 4.6%
NNS		8438		 4.1%
,		8062		 3.9%
CC		6706		 3.3%
VBD		5402		 2.6%
VBP		5374		 2.6%
VBZ		4578		 2.2%
CD		3998		 2.0%
VBN		3967		 1.9%
VBG		3330		 1.6%
MD		3294		 1.6%
TO		3286		 1.6%
PRP$		3068		 1.5%
-RRB-		1008		 0.5%
-LRB-		973		 0.5%
WDT		948		 0.5%
WRB		869		 0.4%
:		866		 0.4%
``		813		 0.4%
''		785		 0.4%
WP		760		 0.4%
RP		755		 0.4%
UH		689		 0.3%
POS		684		 0.3%
HYPH		664		 0.3%
JJR		503		 0.2%
NNPS		498		 0.2%
JJS		383		 0.2%
EX		359		 0.2%
NFP		338		 0.2%
GW		294		 0.1%
ADD		292		 0.1%
RBR		276		 0.1%
$		258		 0.1%
PDT		175		 0.1%
RBS		169		 0.1%
SYM		156		 0.1%
LS		117		 0.1%
FW		93		 0.0%
AFX		48		 0.0%
WP$		15		 0.0%
XX		1		 0.0%


In [22]:
BATCH_SIZE = 128

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE)

In [23]:
# RNN의 향상버전인 Bi-LSTM을 이용하여 모델 생성
class BiLSTMPOSTagger(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx): 
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        
        self.lstm = nn.LSTM(embedding_dim, 
                            hidden_dim, 
                            num_layers = n_layers, 
                            bidirectional = bidirectional,
                            dropout = dropout if n_layers > 1 else 0)
        
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)  
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        # print(text.shape) # [text lengths, batch_size]       
        
        embedded = self.dropout(self.embedding(text))
        # print(embedded.shape) # [text lengths, batch_size, embedding_dim]      

        #outputs holds the backward and forward hidden states in the final layer
        #hidden and cell are the backward and forward hidden and cell states at the final time-step
        
        outputs, (hidden, cell) = self.lstm(embedded)
        # print(outputs.shape) # [text lengths, batch_size, hidden_dim * 2]
        # print(hidden.shape) # [n_layers * 2, batch_size, hidden_dim]
        # print(cell.shape) # [n_layers * 2, batch_size, hidden_dim]
  
        predictions = self.fc(self.dropout(outputs))
        # print(predictions.shape) # [text lengths, batch_size, len(UD_TAG.vocab) == output_dim]
        
        return predictions

In [24]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = len(UD_TAGS.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = BiLSTMPOSTagger(INPUT_DIM, 
                        EMBEDDING_DIM, 
                        HIDDEN_DIM, 
                        OUTPUT_DIM, 
                        N_LAYERS, 
                        BIDIRECTIONAL, 
                        DROPOUT, 
                        PAD_IDX)

In [25]:
# 가중치 초기화
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)
        
model.apply(init_weights)

BiLSTMPOSTagger(
  (embedding): Embedding(8866, 100, padding_idx=1)
  (lstm): LSTM(100, 128, num_layers=2, dropout=0.25, bidirectional=True)
  (fc): Linear(in_features=256, out_features=18, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [26]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,522,010 trainable parameters


In [27]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([8866, 100])


In [28]:
model.embedding.weight.data.copy_(pretrained_embeddings) # 임베딩 벡터값 copy

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.9261,  2.3049,  0.5502,  ..., -0.3492, -0.5298, -0.1577],
        [-0.5972,  0.0471, -0.2406,  ..., -0.9446, -0.1126, -0.2260],
        [-0.4809,  2.5629,  0.9530,  ...,  0.5278, -0.4588,  0.7294]])

In [29]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM) # 1번쨰 임베딩 벡터에 PAD_IDX(0)으로 초기화
# model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM) # 0번째 임베딩 벡터에 UNK_IDX(1)으로 초기화

print(model.embedding.weight.data)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.9261,  2.3049,  0.5502,  ..., -0.3492, -0.5298, -0.1577],
        [-0.5972,  0.0471, -0.2406,  ..., -0.9446, -0.1126, -0.2260],
        [-0.4809,  2.5629,  0.9530,  ...,  0.5278, -0.4588,  0.7294]])


In [30]:
optimizer = optim.Adam(model.parameters())

In [31]:
TAG_PAD_IDX = UD_TAGS.vocab.stoi[UD_TAGS.pad_token] 

# 레이블 데이터의 패딩 토큰은 비용 함수의 연산에 포함시키지도 않도록 레이블 데이터의 패딩 토큰을 무시
criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

In [32]:
# model = model.to(device)
# criterion = criterion.to(device)

In [33]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    배치 당 정확도를 반환 즉, 8/10이 맞으면 8이 아닌 0.8이 반환
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]])

In [34]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [35]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        # iterator : 인코딩 된 text
        text = batch.text
        #print(text.shape) # [text lengths, batch_size]
        tags = batch.udtags
        #print(tags.shape) # [text lengths, batch_size]
        
        optimizer.zero_grad()
             
        predictions = model(text)
        # print(predictions.shape) # [sent len, batch size, output dim]
        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)    
        #predictions = [text lengths * batch size, output dim]
        #tags = [text lengths * batch size]
        
        loss = criterion(predictions, tags)
                
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [36]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch.text
            tags = batch.udtags
            
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [37]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

<ipython-input-33-71845e0bc7a1>:6: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  non_pad_elements = (y != tag_pad_idx).nonzero()


Epoch: 01 | Epoch Time: 0m 47s
	Train Loss: 1.337 | Train Acc: 58.14%
	 Val. Loss: 0.677 |  Val. Acc: 78.71%
Epoch: 02 | Epoch Time: 0m 47s
	Train Loss: 0.481 | Train Acc: 84.90%
	 Val. Loss: 0.508 |  Val. Acc: 83.09%
Epoch: 03 | Epoch Time: 0m 48s
	Train Loss: 0.350 | Train Acc: 88.96%
	 Val. Loss: 0.445 |  Val. Acc: 85.30%
Epoch: 04 | Epoch Time: 0m 47s
	Train Loss: 0.292 | Train Acc: 90.71%
	 Val. Loss: 0.407 |  Val. Acc: 86.23%
Epoch: 05 | Epoch Time: 0m 46s
	Train Loss: 0.253 | Train Acc: 92.02%
	 Val. Loss: 0.397 |  Val. Acc: 86.28%
Epoch: 06 | Epoch Time: 0m 48s
	Train Loss: 0.226 | Train Acc: 92.82%
	 Val. Loss: 0.381 |  Val. Acc: 87.09%
Epoch: 07 | Epoch Time: 0m 49s
	Train Loss: 0.206 | Train Acc: 93.40%
	 Val. Loss: 0.372 |  Val. Acc: 87.45%
Epoch: 08 | Epoch Time: 0m 48s
	Train Loss: 0.192 | Train Acc: 93.89%
	 Val. Loss: 0.362 |  Val. Acc: 87.59%
Epoch: 09 | Epoch Time: 0m 47s
	Train Loss: 0.176 | Train Acc: 94.30%
	 Val. Loss: 0.356 |  Val. Acc: 88.34%
Epoch: 10 | Epoch T

In [38]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.365 |  Test Acc: 88.30%


In [39]:
def tag_sentence(model, sentence, text_field, tag_field): # device,
    
    model.eval()
    
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text for token in nlp(sentence)]
    else:
        tokens = [token for token in sentence]

    if text_field.lower:
        tokens = [t.lower() for t in tokens]
        
    numericalized_tokens = [text_field.vocab.stoi[t] for t in tokens]

    unk_idx = text_field.vocab.stoi[text_field.unk_token]
    
    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1)  #.to(device)
         
    predictions = model(token_tensor)
    
    top_predictions = predictions.argmax(-1)
    
    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]
    
    return tokens, predicted_tags, unks

In [40]:
example_index = 1

sentence = vars(train_data.examples[example_index])['text']
actual_tags = vars(train_data.examples[example_index])['udtags']

print(sentence)

['[', 'this', 'killing', 'of', 'a', 'respected', 'cleric', 'will', 'be', 'causing', 'us', 'trouble', 'for', 'years', 'to', 'come', '.', ']']


In [41]:
tokens, pred_tags, unks = tag_sentence(model, 
#                                        device, 
                                       sentence, 
                                       TEXT, 
                                       UD_TAGS)

print(unks)

['respected', 'cleric']


In [42]:
print("Pred. Tag\tActual Tag\tCorrect?\tToken\n")

for token, pred_tag, actual_tag in zip(tokens, pred_tags, actual_tags):
    correct = '✔' if pred_tag == actual_tag else '✘'
    print(f"{pred_tag}\t\t{actual_tag}\t\t{correct}\t\t{token}")

Pred. Tag	Actual Tag	Correct?	Token

PUNCT		PUNCT		✔		[
DET		DET		✔		this
VERB		NOUN		✘		killing
ADP		ADP		✔		of
DET		DET		✔		a
ADJ		ADJ		✔		respected
NOUN		NOUN		✔		cleric
AUX		AUX		✔		will
AUX		AUX		✔		be
VERB		VERB		✔		causing
PRON		PRON		✔		us
NOUN		NOUN		✔		trouble
ADP		ADP		✔		for
NOUN		NOUN		✔		years
PART		PART		✔		to
VERB		VERB		✔		come
PUNCT		PUNCT		✔		.
PUNCT		PUNCT		✔		]


In [43]:
sentence = 'The Queen will deliver a speech about the conflict in North Korea at 1pm tomorrow.'

tokens, tags, unks = tag_sentence(model, 
#                                   device, 
                                  sentence, 
                                  TEXT, 
                                  UD_TAGS)

print(unks)

[]


In [44]:
print("Pred. Tag\tToken\n")

for token, tag in zip(tokens, tags):
    print(f"{tag}\t\t{token}")

Pred. Tag	Token

DET		the
NOUN		queen
AUX		will
VERB		deliver
DET		a
NOUN		speech
ADP		about
DET		the
NOUN		conflict
ADP		in
PROPN		north
PROPN		korea
ADP		at
NUM		1
NOUN		pm
NOUN		tomorrow
PUNCT		.
